<span style="font-family:Papyrus; font-size:2.5em;">Generative Adversarial Network (GAN) para sillas</span>
---
![](https://espresso-jobs.com/conseils-carriere/wp-content/uploads/2019/05/monalisa.gif)
<br>

# Preparativos





**Conectamos drive**

In [ ]:
#Si trabajamos con carpeta local de drive, primero conectamos
from google.colab import drive
drive.mount('/content/drive')

**Importamos bibliotecas**

In [ ]:
from __future__ import print_function
from time import time
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.notebook import tqdm
import os
from torch.autograd import Variable

print('librerías importadas')

In [ ]:
# NO ejecutar en colab. Solo si trabajamos con kaggle para ver inputs disponibles
!ls /kaggle/input

**Definimos rutas y comprobamos dataset**

In [ ]:
# Descomentar rutas para dataset original si trabajamos con Kaggle
# FOLDER = '/kaggle/input/'
# CLASS = 'antic-chairs/antic_chairs/'
# PATH =  '/kaggle/input/antic-chairs/antic_chairs/'

# Descomentar rutas para dataset original si trabajamos con Colab
# FOLDER = '/content/drive/MyDrive/ColabNotebooks/datasets/'
# CLASS = 'chairs/antic_chairs/'
# PATH = '/content/drive/MyDrive/ColabNotebooks/datasets/chairs/antic_chairs/'

# rutas para datset activo
FOLDER = '/content/drive/MyDrive/ColabNotebooks/dataset3/'
CLASS = 'sillas/sillas_100ext_aum/'
PATH = '/content/drive/MyDrive/ColabNotebooks/dataset3/sillas/sillas_100ext_aum/'

BASE_SAVE_PATH = '/content/drive/MyDrive/ColabNotebooks/results/'
TEST_SAVE_PATH = '/content/drive/MyDrive/ColabNotebooks/test_generated_images/'


# hcermos recuento delvnúmero de imágenes en dataset
images = os.listdir(PATH)
print(f'There are {len(os.listdir(PATH))} chairs .')

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12,10))

for indx, axis in enumerate(axes.flatten()):
    rnd_indx = np.random.randint(0, len(os.listdir(PATH)))
    # https://matplotlib.org/users/image_tutorial.html
    img = plt.imread(PATH + images[rnd_indx])
    imgplot = axis.imshow(img)
    axis.set_title(images[rnd_indx])
    axis.set_axis_off()
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

print('Maximo: ', np.max(img), 'Minimo: ', np.min(img))

**Seleccionamos cuda o cpu**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Preparación de imágenes


**Transformación de imágenes del dataset y dataloader**

In [ ]:
batch_size = 32 # número de imgs de cada lote
image_size = 64 #dimensiones de las imágenes
latent_dim = 100 # dimensionalidad del vector de ruido


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# algoritmo de transformación de imgs con normalización
transform = transforms.Compose([transforms.Resize(64),
                                transforms.CenterCrop(64),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# algoritmo de transformación de imgs sin normalización
transform_noresize = transforms.Compose([transforms.Resize(64),
                                        #transforms.CenterCrop(64),
                                         transforms.ToTensor()])

# Definimos cargador 1 de imágenes para entrenamiento
train_data = datasets.ImageFolder(FOLDER, transform=transform)
train_loader = DataLoader(train_data, shuffle=True,
                                           batch_size=batch_size)

# Definimos cargador 2 de imágenes para entrenamiento
train_data_noresize = datasets.ImageFolder(FOLDER, transform=transform_noresize)
train_loader_noresize = DataLoader(train_data_noresize, shuffle=True,
                                           batch_size=batch_size)

**Comprobamos los train_loader del dataset**

In [ ]:
# Definimos imágenes con cada train_loader
imgs, label = next(iter(train_loader))
imgs = imgs.numpy().transpose(0, 2, 3, 1)

imgs_noresize, label_noresize = next(iter(train_loader_noresize))
imgs_noresize = imgs_noresize.numpy().transpose(0, 2, 3, 1)

# Mostramos por pantalla las imágenes de ambos cargadores
print(imgs.shape)
for i in range(5):
    plt.imshow(imgs[i])
    plt.show()

print(imgs_noresize.shape)
for i in range(5):
    plt.imshow(imgs_noresize[i])
    plt.show()

# Definimos pesos iniciales


In [ ]:
def weights_init(m):
    """
    Takes as input a neural network m that will initialize all its weights.
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# Generadores

Ejecutamos uno de los 3 siguientes Generadores

In [ ]:
# GENERADOR 1 (generador inicial)

class G(nn.Module):
    def __init__(self):

        super(G, self).__init__()

        self.main = nn.Sequential(  # Definimos secuencia de módulos (capas, operaciones)
                nn.ConvTranspose2d(latent_dim, 512, 4, stride=2, padding=0, bias=False),
                nn.BatchNorm2d(512),
                nn.ReLU(True),
                nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1, bias=False),
                nn.BatchNorm2d(256),
                nn.ReLU(True),
                nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(True),
                nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(True),
                nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1, bias=False),
                nn.Sigmoid()
                )

# Definimos flujo de datos
    def forward(self, input):
        output = self.main(input)
        return output

# Definimos Generador
netG = G().to(device)
netG.apply(weights_init)

In [ ]:
# GENERADOR 2 CON NÚMERO DE FILTROS CONFIGURABLE

latent_dim = 100 # dimensión de vector ruido de entrada
ngf = 128 # factor de número de filtros para bloques deconv

class G(nn.Module):
    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(  # Definimos secuencia de módulos (capas, operaciones)

            #1 (desde vector de ruido)
            # Input: (batch_size, latent_dim, 1, 1)
            nn.ConvTranspose2d(latent_dim, ngf * 8, 4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # Output: (batch_size, ngf * 8, 4, 4)

            #2
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # Output: (batch_size, ngf * 4, 8, 8)

            #3
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # Output: (batch_size, ngf * 2, 16, 16)

            #4
            nn.ConvTranspose2d(ngf * 2, ngf, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # Output: (batch_size, ngf, 32, 32)

            #5 (Final layer for 64x64 output)
            nn.ConvTranspose2d(ngf, 3, 4, stride=2, padding=1, bias=False),
            nn.Sigmoid()
            # Output: (batch_size, 3, 64, 64)
        )

    def forward(self, input):
        return self.main(input)


# Definimos flujo de datos
    def forward(self, input):
        output = self.main(input)
        return output

# Definimos Generador
netG = G().to(device)
netG.apply(weights_init)

In [ ]:
# GENERADOR 3 (6 BLOQUES DECONVOLUCIONALES)

latent_dim = 100 # dimensión de vector ruido de entrada

ngf = 64 # factor de número de filtros para bloques deconv

class G(nn.Module):
    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential( # Definimos secuencia de módulos (capas, operaciones)

            # 1: latent_dim -> 4x4 (ngf*16 filtros, canales)
            # Input: (batch_size, latent_dim, 1, 1)
            nn.ConvTranspose2d(latent_dim, ngf * 16, 4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            # Output: (batch_size, ngf * 16, 4, 4)

            # 2: 4x4 -> 8x8 (ngf*8 filtros, canales)
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # Output: (batch_size, ngf * 8, 8, 8)

            # 3: 8x8 -> 16x16 (ngf*4 filtros, canales)
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # Output: (batch_size, ngf * 4, 16, 16)

            # 4: 16x16 -> 32x32 (ngf*2 filtros, canales)
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # Output: (batch_size, ngf * 2, 32, 32)

            # 5: 32x32 -> 64x64 (ngf canales) --- Esta capa alcanza la dimensión de imagen deseada (64 x 64)
            nn.ConvTranspose2d(ngf * 2, ngf, 4, stride=2, padding=1, bias=False), # (32-1)*2 - 2*1 + 4 = 62-2+4 = 64
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # Output: (batch_size, ngf, 64, 64)

            # 6: 64x64 -> 64x64 RGB (3 canales) --- Esta capa modifica el número de canales pero no dimensión del mapa
            nn.ConvTranspose2d(ngf, 3, 1, stride=1, padding=0, bias=False), # kernel_size=1, stride=1, padding=0 is like 1x1 conv
            nn.Sigmoid()

            # Output: (batch_size, 3, 64, 64)
        )

# Definimos flujo de datos
   # def forward(self, input):
       # return self.main(input)
def forward(self, input):
        output = self.main(input)
        return output

# Creamos Generador
netG = G().to(device)
netG.apply(weights_init)

# Discriminador

In [ ]:

class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential( # Definimos secuencia de módulos (capas, operaciones)

                # 1
                nn.Conv2d(3, 64, 4, stride=1, padding=1, bias=False),
                nn.MaxPool2d(2,2),
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Dropout(0.3),
                # 2
                nn.Conv2d(64, 128, 4, stride=1, padding=1, bias=False),
                nn.MaxPool2d(2,2),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Dropout(0.3),
                # 3
                nn.Conv2d(128, 256, 4, stride=1, padding=1, bias=False),
                nn.MaxPool2d(2,2),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Dropout(0.3),
                # 4
                nn.Conv2d(256, 512, 4, stride=1, padding=1, bias=False),
                nn.MaxPool2d(2,2),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Dropout(0.3),

                nn.Flatten(),

                # 5. Red densa
                nn.Linear(4608,100),
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Linear(100,1),
                nn.Sigmoid()
                )

# Definimos flujo de datos
    def forward(self, input):
        output = self.main(input)
        # .view(-1) = Aplana salida a 1D en lugar de 2D
        return output.view(-1)


# Creamos el Discriminador
netD = D().to(device)
netD.apply(weights_init)


# Entrenamiento

In [ ]:
# NO ejecutar en colab
!mkdir results
!ls

In [ ]:
LR = 0.0002 # tasa de aprendizaje

# tasas diferenciadas para G y D
LR_G = 0.0002
LR_D = 0.0002

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=LR_D, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=LR_G, betas=(0.5, 0.999))


**Seleccionar y ejecutar uno de los 2 siguientes entrenamientos**

In [ ]:
# ENTRENADOR 1 (una optimización de generador por cada una del discriminador, mismo vector de ruido)

# Descomentar el data loader seleccionado:
# data_loader = train_loader
data_loader = train_loader_noresize

EPOCH = 300 # número de épocas

losses_D = []
losses_G = []

t0 = time()
for epoch in range(EPOCH):
    epoch_loss_D = 0
    epoch_loss_G = 0
    num_batches = 0
    for i, data in enumerate(data_loader, 0):

        # Actualizamos pesos d ela red del Discriminador
        netD.zero_grad()

        # Entrenamos Dscriminador con lote de imágenes reales
        input,_ = data
        input = input.to(device)
        target = torch.ones(input.size()[0]).to(device)
        output = netD(input)
        errD_real = criterion(output, target)

        # Entrenamos Discriminador con lote de imágenes falsas producidad por el Generador
        noise = torch.randn(input.size()[0], 100, 1, 1).to(device)
        fake = netG(noise)
        target = torch.zeros(input.size()[0]).to(device)
        output = netD(fake.detach())
        errD_fake = criterion(output, target)

        # Sumamos errores y hacemos bacpropagation
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        # Actualizamos (optimizamos) pesos del Generador
        netG.zero_grad()
        target = torch.ones(input.size()[0]).to(device)
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()

        # Acumulamos errores
        epoch_loss_D += errD.item()
        epoch_loss_G += errG.item()
        num_batches += 1

    losses_D.append(epoch_loss_D / num_batches)
    losses_G.append(epoch_loss_G / num_batches)

    # Mostraos errores por pantalla y guardamos muestra de imágenes reales y otra de imñagenes generadas
    print('[%d/%d] Loss_D: %.4f; Loss_G: %.4f' % (epoch, EPOCH, errD.item(), errG.item()))
    save_image(input, os.path.join(BASE_SAVE_PATH, 'real_samples.png'), normalize=True)
    save_image(fake.data, os.path.join(BASE_SAVE_PATH, 'fake_samples_epoch_%03d.png' % epoch), normalize=True)
tf = time()
print(tf - t0)

# Guardamos pesos de Generador y Discriminador
best_model_g = os.path.join(BASE_SAVE_PATH, 'generator_v2.pth')
best_model_d = os.path.join(BASE_SAVE_PATH, 'discriminator_v2.pth')
torch.save(netG.state_dict(), best_model_g)
torch.save(netD.state_dict(), best_model_d)

# Generamos y guardamos gráfica evolución de errores
plt.figure(figsize=(10, 5))
plt.plot(range(EPOCH), losses_D, label='Perdida Discriminador', color='red')
plt.plot(range(EPOCH), losses_G, label='Perdida Generador', color='blue')
plt.xlabel('Época')
plt.ylabel('Perdida')
plt.title('Perdida del Generador y Discriminador por Época')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(BASE_SAVE_PATH, 'loss_plot.png')) # Guarda la gráfica
plt.show()

# Generamos lote de imágenes falsas

if not os.path.exists(TEST_SAVE_PATH):
    os.makedirs(TEST_SAVE_PATH)
    print(f"Carpeta de imágenes generadas creada en: {TEST_SAVE_PATH}")

im_batch_size = 50
n_images=10

for i_batch in tqdm(range(0, n_images, im_batch_size)):
    gen_z = torch.randn(im_batch_size, 100, 1, 1, device=device)
    gen_images = netG(gen_z)
    images = gen_images.to("cpu").clone().detach()
with torch.no_grad():
    outputs = netD(gen_images)
    images = images.numpy().transpose(0, 2, 3, 1)
    for i_image in range(gen_images.size(0)):
        save_image(gen_images[i_image, :, :, :], os.path.join(TEST_SAVE_PATH, f'image_{i_batch+i_image:05d}.png'))

for i_image in range(gen_images.size(0)):
        score = outputs[i_image].item()
        print(f'Imagen {i_batch + i_image:05d} - Discriminador: {score:.4f}')
        save_image(gen_images[i_image, :, :, :], os.path.join(TEST_SAVE_PATH, f'image_{i_batch+i_image:05d}.png'))

fig = plt.figure(figsize=(25, 16))

# mostramos 10 imagenes falsas por clase
for i, j in enumerate(images[:32]):
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
    plt.imshow(j)

In [ ]:
# ENTRENADOR 2 ( 2 optimizaciones de generador por cada una del discriminador, vectores ruido nuevos)

# Descomentar el data loader seleccionado:
# data_loader = train_loader
data_loader = train_loader_noresize

EPOCH = 600 # número de épocas

losses_D = []
losses_G = []

t0 = time()
for epoch in range(EPOCH):
    epoch_loss_D = 0
    epoch_loss_G = 0
    num_batches = 0
    for i, data in enumerate(data_loader, 0):

        #Actualizamos pesos de la red del Discrimnador
        netD.zero_grad()

        # Entrenamos Discriminador con imágenes reales del dataset
        input,_ = data
        input = input.to(device)
        target = torch.ones(input.size()[0]).to(device)
        output = netD(input)
        errD_real = criterion(output, target)

        # Entrenamos el Discriminador con imágenes falsas producidas por el Generador
        noise = torch.randn(input.size()[0], 100, 1, 1).to(device)
        fake = netG(noise)
        target = torch.zeros(input.size()[0]).to(device)
        output = netD(fake.detach())
        errD_fake = criterion(output, target)

        # Calculamos error total del Discriminador y hacemos backpropagation
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        # Hacemos primera actualización de pesos del Generador
        # con nuevo oote de imágenes sintéticas
        noise = torch.randn(input.size()[0], 100, 1, 1).to(device)
        fake = netG(noise)
        netG.zero_grad()
        target = torch.ones(input.size()[0]).to(device)
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()

        # Hacemos segunda actualización de pesos del Generador con nuevo lote de imágenes sintéticas
        noise = torch.randn(input.size()[0], 100, 1, 1).to(device)
        fake = netG(noise)
        netG.zero_grad()
        target = torch.ones(input.size()[0]).to(device)
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()

        # Acumulamos errores y número de lotes por época
        epoch_loss_D += errD.item()
        epoch_loss_G += errG.item()
        num_batches += 1

    losses_D.append(epoch_loss_D / num_batches)
    losses_G.append(epoch_loss_G / num_batches)

    # 3rd Step: Printing the losses and saving the real images and the generated images of the last minibatch
    # Mostramos los errores del G y D en la época y generamos una muestra de imágenes
    print('[%d/%d] Loss_D: %.4f; Loss_G: %.4f' % (epoch, EPOCH, errD.item(), errG.item()))
    save_image(input, os.path.join(BASE_SAVE_PATH, 'real_samples.png'), normalize=True)
    save_image(fake.data, os.path.join(BASE_SAVE_PATH, 'fake_samples_epoch_%03d.png' % epoch), normalize=True)

tf = time()
print(tf - t0)

# Salvamos pesos de Generador y Discriminador
best_model_g = os.path.join(BASE_SAVE_PATH, 'generator_v2.pth')
best_model_d = os.path.join(BASE_SAVE_PATH, 'discriminator_v2.pth')
torch.save(netG.state_dict(), best_model_g)
torch.save(netD.state_dict(), best_model_d)

# generamos y guardamos gráfica de errores
plt.figure(figsize=(10, 5))
plt.plot(range(EPOCH), losses_D, label='Pérdida Discriminador', color='red')
plt.plot(range(EPOCH), losses_G, label='Pérdida Generador', color='blue')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.title('Pérdida del Generador y Discriminador por Época')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(BASE_SAVE_PATH, 'loss_plot.png')) # Guarda la gráfica
plt.show()

# Generamos lote de imágenes de muestra

if not os.path.exists(TEST_SAVE_PATH):
    os.makedirs(TEST_SAVE_PATH)
    print(f"Carpeta de imágenes generadas creada en: {TEST_SAVE_PATH}")

im_batch_size = 50
n_images=10

for i_batch in tqdm(range(0, n_images, im_batch_size)):
    gen_z = torch.randn(im_batch_size, 100, 1, 1, device=device)
    gen_images = netG(gen_z)
    images = gen_images.to("cpu").clone().detach()
with torch.no_grad():
    outputs = netD(gen_images)
    images = images.numpy().transpose(0, 2, 3, 1)
    for i_image in range(gen_images.size(0)):
        save_image(gen_images[i_image, :, :, :], os.path.join(TEST_SAVE_PATH, f'image_{i_batch+i_image:05d}.png'))

for i_image in range(gen_images.size(0)):
        score = outputs[i_image].item()
        print(f'Imagen {i_batch + i_image:05d} - Discriminador: {score:.4f}')
        save_image(gen_images[i_image, :, :, :], os.path.join(TEST_SAVE_PATH, f'image_{i_batch+i_image:05d}.png'))

fig = plt.figure(figsize=(25, 16))

# Mostramos 10 imágenes de cada clase

for i, j in enumerate(images[:32]):
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
    plt.imshow(j)

# OPERACIONES SEPARADAS

# Guardar pesos

In [ ]:
# Guardamos pesos ajustados
best_model_g = os.path.join(BASE_SAVE_PATH, 'generator_v1.pth') # Puedes usar el mismo BASE_SAVE_PATH
best_model_d = os.path.join(BASE_SAVE_PATH, 'discriminator_v1.pth') # Puedes usar el mismo BASE_SAVE_PATH
torch.save(netG.state_dict(), best_model_g)
torch.save(netD.state_dict(), best_model_d)

# Recuperar pesos

In [ ]:
# Cagarmos pesos desde modelo entrenado y guardado
RECOVER_PATH = '/content/drive/MyDrive/ColabNotebooks/ALMACEN/entrenamiento42/' # ruta donde se guardó
best_model_g = os.path.join(RECOVER_PATH, 'generator_42.pth') # nombre del archivo de pesos del generador
best_model_d = os.path.join(RECOVER_PATH, 'discriminator_42.pth') # nombre del archivo de pesos del discriminador

if os.path.exists(best_model_g) and os.path.exists(best_model_d):
    netG.load_state_dict(torch.load(best_model_g, map_location=device)) # Añadir map_location=device si es necesario
    netD.load_state_dict(torch.load(best_model_d, map_location=device)) # Añadir map_location=device si es necesario
    print("Modelos cargados exitosamente desde Drive.")

# netG.load_state_dict(torch.load(best_model_g, weights_only=True))
# netD.load_state_dict(torch.load(best_model_d, weights_only=True))

# Gráfica de errores


In [ ]:
def plot_loss (G_losses, D_losses, epoch):
    plt.figure(figsize=(10,5))
    plt.title("Generator and Discriminator Loss - EPOCH "+ str(epoch))
    plt.plot(G_losses,label="G")
    plt.plot(D_losses,label="D")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

# Mostrar imágenes generadas


In [ ]:
def show_generated_img(n_images=5):
    sample = []
    for _ in range(n_images):
        noise = torch.randn(1, nz, 1, 1, device=device)
        gen_image = netG(noise).to("cpu").clone().detach().squeeze(0)
        gen_image = gen_image.numpy().transpose(1, 2, 0)
        sample.append(gen_image)

    figure, axes = plt.subplots(1, len(sample), figsize = (64,64))
    for index, axis in enumerate(axes):
        axis.axis('off')
        image_array = sample[index]
        axis.imshow(image_array)

    plt.show()
    plt.close()

In [ ]:
nz=100
show_generated_img()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(EPOCH), losses_D, label='Perdida Discriminador', color='red')
plt.plot(range(EPOCH), losses_G, label='Perdida Generador', color='blue')
plt.xlabel('Época')
plt.ylabel('Perdida')
plt.title('Perdida del Generador y Discriminador por Época')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(BASE_SAVE_PATH, 'loss_plot.png'))  # Guarda la gráfica
plt.show()

# Generación de ejemplos

In [ ]:
if not os.path.exists('./test'):
    os.mkdir('./test')

im_batch_size = 50
n_images=10

for i_batch in tqdm(range(0, n_images, im_batch_size)):
    gen_z = torch.randn(im_batch_size, 100, 1, 1, device=device)
    gen_images = netG(gen_z)
    images = gen_images.to("cpu").clone().detach()
with torch.no_grad():
    outputs = netD(gen_images)
    images = images.numpy().transpose(0, 2, 3, 1)
    for i_image in range(gen_images.size(0)):
        save_image(gen_images[i_image, :, :, :], os.path.join('./test', f'image_{i_batch+i_image:05d}.png'))


In [ ]:
    for i_image in range(gen_images.size(0)):
        score = outputs[i_image].item()
        print(f'Imagen {i_batch + i_image:05d} - Discriminador: {score:.4f}')
        save_image(gen_images[i_image, :, :, :], os.path.join('./test', f'image_{i_batch+i_image:05d}.png'))


In [ ]:
fig = plt.figure(figsize=(25, 16))
# Mostrar 10 imágenes de cada clase
for i, j in enumerate(images[:32]):
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
    plt.imshow(j)

In [ ]:
# Generar lote de imágenes (código agrupado)

if not os.path.exists(TEST_SAVE_PATH):
    os.makedirs(TEST_SAVE_PATH)
    print(f"Carpeta de imágenes generadas creada en: {TEST_SAVE_PATH}")

im_batch_size = 50
n_images=10

for i_batch in tqdm(range(0, n_images, im_batch_size)):
    gen_z = torch.randn(im_batch_size, 100, 1, 1, device=device)
    gen_images = netG(gen_z)
    images = gen_images.to("cpu").clone().detach()
with torch.no_grad():
    outputs = netD(gen_images)
    images = images.numpy().transpose(0, 2, 3, 1)
    for i_image in range(gen_images.size(0)):
        save_image(gen_images[i_image, :, :, :], os.path.join(TEST_SAVE_PATH, f'image_{i_batch+i_image:05d}.png'))

for i_image in range(gen_images.size(0)):
        score = outputs[i_image].item()
        print(f'Imagen {i_batch + i_image:05d} - Discriminador: {score:.4f}')
        save_image(gen_images[i_image, :, :, :], os.path.join(TEST_SAVE_PATH, f'image_{i_batch+i_image:05d}.png'))

fig = plt.figure(figsize=(25, 16))

# display 10 images from each class
for i, j in enumerate(images[:32]):
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
    plt.imshow(j)